# Исследование надёжности заёмщиков

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
data = pd.read_csv('data.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## 2. Предобработка данных

In [ ]:
data.info()

### Данные о количестве детей заемщика

In [3]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В данных присутствуют отрицательные значения количества детей у заемщика

In [4]:
children_percent = data.loc[data['children'] < 0, 'children'].count() / data.shape[0]
print(f'Процент ошибочно заполненных данных о количестве детей: {children_percent:.2%}')

Процент ошибочно заполненных данных о количестве детей: 0.22%


In [5]:
median_children = int(data[data['children'] >= 0]['children'].median())
data.loc[data['children'] < 0, 'children'] = median_children
children_percent = data.loc[data['children'] < 0, 'children'].count() / data.shape[0]
print(f'Процент ошибочно заполненных данных о количестве детей: {children_percent:.2%}')

Процент ошибочно заполненных данных о количестве детей: 0.00%


**Вывод:** Пропущенные значения о количестве детей отсутствуют, но есть менее 1% некорректно заполненных данных. Наиболее вероятная причина появления таких данных - это ошибка ввода данных оператором.
Ошибочные данные о количестве детей заменены значением медианы, на случай изменения среднего значения в будущих наборах данных.

### Возраст клиента

In [6]:
dob_years_percent = data[data['dob_years'] == 0]['dob_years'].count() / data.shape[0]
print(f'Процент не заполненных данных возраста клиента: {dob_years_percent:.2%}')

data.loc[data['dob_years'] == 0, 'dob_years'] = int(data['dob_years'].median())

Процент не заполненных данных возраста клиента: 0.47%


**Вывод:** Некоррентно заполнено менее 1% данных о возрасте клиентов. Наиболее вероятная причина появления таких данных - это ошибка ввода данных оператором.
Ошибочные данные о количестве детей заменены значением медианы.

### Данные об общем трудовом стаже в днях

In [7]:
negative_days_employed = data.loc[data['days_employed'] < 0, 'days_employed'].count() / data.shape[0]
null_days_empolyed = data.loc[data['days_employed'] == 0, 'days_employed'].count() / data.shape[0]
positive_days_empolyed = data.loc[data['days_employed'] > 0, 'days_employed'].count() / data.shape[0]

print(f'Процент отрицательных дней трудового стажа: {negative_days_employed:.2%}')
print(f'Процент нулевых дней трудового стажа: {null_days_empolyed:.2%}')
print(f'Процент положительный дней трудового стажа: {positive_days_empolyed:.2%}')

mean_positive_days = data.loc[data['days_employed'] > 0, 'days_employed'].median()
mean_negative_days = data.loc[data['days_employed'] < 0, 'days_employed'].median()

print(f'Средний трудовой стаж в днях положительных значений: {mean_positive_days:.0f}')

Процент отрицательных дней трудового стажа: 73.90%
Процент нулевых дней трудового стажа: 0.00%
Процент положительный дней трудового стажа: 16.00%
Средний трудовой стаж в днях положительных значений: 365213


In [8]:
data.loc[data['days_employed'] > 0, 'income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

In [9]:
data.loc[data['days_employed'] < 0, 'income_type'].value_counts()

сотрудник          10014
компаньон           4577
госслужащий         1312
предприниматель        1
студент                1
в декрете              1
Name: income_type, dtype: int64

In [10]:
data.loc[data['days_employed'] < 0, 'days_employed'] = data.loc[data['days_employed'] < 0, 'days_employed'].abs()
data.loc[data['days_employed'] > 0, 'days_employed'] = 0

**Вывод:** Положительные значения данных трудового стажа имеют слишком большое значение. Средний трудовой стаж в днях положительных значений 365213 дней. Так же данные принадлежат пенсионерам и безработным, следовательно не имеют значения в данном исследовании и могут быть обнулены. Пропущенные значения 

Большая часть значений имеет отрицательное число, что указывает на возможную ошибку в обработке данных. Но абсолютное значение пригодно для обработки.

### Данные семейного статуса и образования

In [11]:
data['family_status'] = data['family_status'].str.lower()
data['education'] = data['education'].str.lower()
data['gender'] = data['gender'].str.lower()

In [12]:
data.loc[(data['gender'] != 'f')& (data['gender'] != 'm'),'gender'] = 'non-bin'

Данные семейного положнния, образования и пола переведены в нижний регистр для приведения к единому виду. Обнаружил клиента неопределенного пола, изменил значение 'XNA' на более общепринятое. 

### Замена вещественного на целочисленный тип

In [13]:
def convert_to_int(x):
    try:
        return int(x)
    except:
        return 0
    
data['days_employed'] = data['days_employed'].apply(convert_to_int)
data['total_income'] = data['total_income'].apply(convert_to_int)

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Замена целочисленного типа на логический в задолженности клиента

In [15]:
data['debt'] = data['debt'].astype('bool')

При изменение типа данных, некорректные строки заменены нулевыми значениям

### Удаление дубликатов данных

In [16]:
data = data.drop_duplicates().reset_index(drop = True)

Предполагаю что дубликаты появились из-за того что в изначальной базе данны были личные данные заемщиков. В исследовании они не являются необходими данными.

### Леммизация целей получения кредита

In [17]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [18]:
m = Mystem()

def lemmatize_purpose(row):
    lemma = m.lemmatize(row)
        
    if 'жилье' in lemma or 'недвижимость' in lemma:
        return 'ипотека'
    elif 'автомобиль' in lemma:
        return 'автокредит'
    elif 'образование' in lemma:
        return 'кредит на образование'
    elif 'свадьба' in lemma:
        return 'кредит на свадьбу'        
    else:
        return 'потребительский'

purpose_dict = pd.DataFrame([], columns=['purpose_id', 'purpose_type'])   
purpose_dict['purpose_type'] = data['purpose'].apply(lemmatize_purpose)
purpose_dict = purpose_dict.drop_duplicates().reset_index(drop=True)
purpose_dict['purpose_id'] = purpose_dict.index

In [19]:
def purpose_id(row):
    return int(purpose_dict[purpose_dict['purpose_type'] == row]['purpose_id'])

data['purpose_type'] = data['purpose'].apply(lemmatize_purpose)
data['purpose_id'] = data['purpose_type'].apply(purpose_id)

### Категоризация данных

In [20]:
education_dict = pd.DataFrame(data={'education': data['education'].value_counts().keys().tolist(), 'education_id': data['education_id'].value_counts().keys().tolist()})
print(education_dict)

             education  education_id
0              среднее             1
1               высшее             0
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4


Определил словарь классификации уровня образования для исследования зависимости образования и возврата кредита.

In [21]:
family_dict = pd.DataFrame(data={'family_status': data['family_status'].value_counts().keys().tolist(), 'family_status_id': data['family_status_id'].value_counts().keys().tolist()})
print(family_dict)

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2  не женат / не замужем                 4
3              в разводе                 3
4         вдовец / вдова                 2


Определил словарь классификации семейного положения для исследования зависимости семейного статуса и возврата кредита.

In [22]:
def children_group(children_count):
    if children_count < 1:
        return 'нет детей'
    elif children_count == 1:
        return 'один ребенок'
    elif 1 < children_count < 3:
        return 'среднее количество детей'
    else:
        return 'много детей'
    
children_dict = pd.DataFrame([], columns=['children_id', 'children_group'])    
children_dict['children_group'] = data['children'].apply(children_group)
children_dict = children_dict.drop_duplicates().reset_index(drop=True)
children_dict['children_id'] = children_dict.index

In [23]:
children_dict

,children_id,children_group
0,0,один ребенок
1,1,нет детей
2,2,много детей
3,3,среднее количество детей


In [24]:
def children_id(row):
    return int(children_dict[children_dict['children_group'] == row]['children_id'])

data.insert(1, 'children_id', None)
data.insert(2, 'children_group', None)

data['children_group'] = data['children'].apply(children_group)
data['children_id'] = data['children_group'].apply(children_id)

Определил словарь классификации количества детей для исследования зависимости наличия детей и возврата кредита. 

In [25]:
def total_income_dict(income):
    if income <= 12000:
        return 'низкий уровень'
    elif 12000 < income <= 35000:
        return 'ниже среднего'
    elif 35000 < income <= 50000:
        return 'средний уровень'
    elif 50000 < income <= 100000:
        return 'выше среднего'
    else:
        return 'высокий уровень'
    

income_dict = pd.DataFrame([], columns=['total_income_id', 'total_income_group'])    
income_dict['total_income_group'] = data['total_income'].apply(total_income_dict)
income_dict = income_dict.drop_duplicates().reset_index(drop=True)
income_dict['total_income_id'] = income_dict.index

In [27]:
data.insert(13, 'total_income_id', None)
data.insert(12, 'total_income_group', None)

data['total_income_group'] = data['total_income'].apply(total_income_dict)

In [28]:
def total_income_id(row):
    return int(income_dict[income_dict['total_income_group'] == row]['total_income_id'])

data['total_income_id'] = data['total_income_group'].apply(total_income_id)

Определил словарь классификации уровня дохода для исследования зависимости уровня дохода и возврата кредита. 

## 3. Ответ на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок? 

In [30]:
children_pivot = data.pivot_table(index=['children_group'], columns='debt', values='children_id', aggfunc='count')
children_pivot['children_ratio'] = children_pivot[True] / (children_pivot[False] + children_pivot[True])
print(children_pivot.sort_values(by='children_ratio'))

debt                      False  True  children_ratio
children_group                                       
нет детей                 13073  1064        0.075263
много детей                 417    39        0.085526
один ребенок               4364   444        0.092346
среднее количество детей   1858   194        0.094542


**Вывод:** Наибольший процент возрата по кредиту среди бездетных клиентов. На втором месте многодетные клиенты. У бездетных клиентов меньше расходов на детей, многодетным выплачиваются пособия.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [31]:
family_pivot = data.pivot_table(index=['family_status'], columns='debt', values='family_status_id', aggfunc='count')
family_pivot['family_ratio'] = family_pivot[True] / (family_pivot[False] + family_pivot[True])
print(family_pivot.sort_values(by='family_ratio'))

debt                   False  True  family_ratio
family_status                                   
вдовец / вдова           896    63      0.065693
в разводе               1110    85      0.071130
женат / замужем        11408   931      0.075452
гражданский брак        3762   388      0.093494
не женат / не замужем   2536   274      0.097509


**Вывод:** Наибольший процент возрата по кредиту среди вдов. На втором месте разведенные клиенты.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [32]:
total_income_pivot = data.pivot_table(index=['total_income_group'], columns='debt', values='total_income_id', aggfunc='count')
total_income_pivot['total_income_ratio'] = total_income_pivot[True] / (total_income_pivot[True] + total_income_pivot[False])
print(total_income_pivot.sort_values(by='total_income_ratio'))

debt                False  True  total_income_ratio
total_income_group                                 
ниже среднего          66     3            0.043478
средний уровень       283    20            0.066007
низкий уровень       1932   170            0.080875
выше среднего        3760   331            0.080909
высокий уровень     13671  1217            0.081744


**Вывод:** Наибольший процент возрата по кредиту у клиентов с доходом ниже среднего. Возможная причина в том что либо указан официальный доход, либо были использованы льготные условия для кредита

### Как разные цели кредита влияют на его возврат в срок?

In [37]:
purpose_pivot = data.pivot_table(index=['purpose_type'], columns='debt', values='purpose_id', aggfunc='count')
purpose_pivot['purpose_ratio'] = purpose_pivot[True] / (purpose_pivot[True] + purpose_pivot[False])
print(purpose_pivot.sort_values(by='purpose_ratio'))

debt                   False  True  purpose_ratio
purpose_type                                     
ипотека                10029   782       0.072334
кредит на свадьбу       2137   186       0.080069
кредит на образование   3643   370       0.092200
автокредит              3903   403       0.093590


**Вывод:** Наиболее надежные клиенты среди тех кто берет ипотеку и кредит на свадьбу. Предполагаю что к ипотеке клиенты более подходят ответственно и ипотечные кредиты имеют более длительный срок. На втором месте кредиты на свадьбу, очевидно что выплачивать кредит проще не одному.

## 4. Вывод

В процессе предобработки данных выявлена аномалии в данных трудового стажа (отрицательные и слишком большие значения). Чтобы можно было их использовать в анализе нужно выяснить их причину. \
Так же определен количества детей, семейного положения, уровня дохода и целей кредита на возврат его в срок.